In [31]:
!python -V

Python 3.11.7


In [32]:
import pandas as pd

In [33]:
import pickle

In [34]:
#import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [35]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [36]:
import mlflow

In [37]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [38]:
mlflow.set_tracking_uri("sqlite:///mlflow1.db")
mlflow.set_experiment("nyc-taxi-experiment_1")

<Experiment: artifact_location='file:///c:/Users/mikes/Documents/STUDY/mlops-zoomcamp/mlops_zoomcamp_hw/02-experiment-tracking/mlruns/2', creation_time=1747461996647, experiment_id='2', last_update_time=1747461996647, lifecycle_stage='active', name='nyc-taxi-experiment_1', tags={}>

In [39]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
df_val['duration'] = (df_val['lpep_dropoff_datetime'] - df_val['lpep_pickup_datetime']).dt.total_seconds() / 60
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]

In [40]:
#df = pd.read_parquet('./data/green_tripdata_2021-01.parquet')

df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')

df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

df["weekday"] = df.lpep_pickup_datetime.dt.weekday
df["hour"] =  df.lpep_pickup_datetime.dt.hour

categorical = ['PULocationID', 'DOLocationID',  'weekday', 'hour']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [41]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,weekday,hour
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.0,None,0.3,6.80,2.0,1.0,0.00,3.933333,4,0
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,0.0,None,0.3,16.86,1.0,1.0,2.75,8.750000,4,0
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,0.0,None,0.3,8.30,1.0,1.0,0.00,5.966667,4,0
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.0,None,0.3,9.30,2.0,1.0,0.00,7.083333,3,23
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.5,...,0.0,None,0.3,5.76,1.0,1.0,0.00,2.316667,4,0


In [42]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print("Feature matrix dimensions:", X_train.shape)

Feature matrix dimensions: (73908, 538)


In [43]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred)

95.25045983499173

In [44]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [16]:
#df_train = read_dataframe('./data/green_tripdata_2023-01.parquet')
#df_val = read_dataframe('./data/green_tripdata_2023-02.parquet')

In [45]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
df_train.to_parquet('data/green_tripdata_2021-01.parquet')
df_val.to_parquet('data/green_tripdata_2021-02.parquet')

In [46]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [47]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [48]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer", "johndoe")
    mlflow.log_param("train-data-path", "green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "green_tripdata_2021-02.parquet")
    alpha = 0.1
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)

    rmse= np.sqrt(mean_squared_error(y_train, y_pred))
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact('models/lin_reg.bin', artifact_path='models_pickle')
    # saving model as artifact

In [49]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:

params = {
    'max_depth': 60,
    'learning_rate': 0.10027586753294233,
    'reg_lambda': 0.20074980286498087,
    'reg_alpha': 0.15788520719314586,
    'min_child_weight': 0.5999999488650849,
    'objective': 'reg:linear',
    'seed': 42
}

mlflow.xgboost.autolog()
booster = xgb.train(
            params = params, 
            dtrain = train, 
            num_boost_round = 1000, 
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )


2025/05/17 08:55:49 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '07e900f654a7428f9429563732827c67', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
c:\Users\mikes\Documents\STUDY\mlops-zoomcamp\mlops_zoomcamp_hw\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [08:56:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.40045
[1]	validation-rmse:10.69886
[2]	validation-rmse:10.09119
[3]	validation-rmse:9.56644
[4]	validation-rmse:9.11606
[5]	validation-rmse:8.73345
[6]	validation-rmse:8.40320
[7]	validation-rmse:8.12719
[8]	validation-rmse:7.89416
[9]	validation-rmse:7.69017
[10]	validation-rmse:7.51987
[11]	validation-rmse:7.37998
[12]	validation-rmse:7.26015
[13]	validation-rmse:7.15713
[14]	validation-rmse:7.06991
[15]	validation-rmse:6.99932
[16]	validation-rmse:6.93628
[17]	validation-rmse:6.88540
[18]	validation-rmse:6.84025
[19]	validation-rmse:6.80301
[20]	validation-rmse:6.76917
[21]	validation-rmse:6.73974
[22]	validation-rmse:6.71626
[23]	validation-rmse:6.69248
[24]	validation-rmse:6.67335
[25]	validation-rmse:6.65691
[26]	validation-rmse:6.63945
[27]	validation-rmse:6.62689
[28]	validation-rmse:6.61684
[29]	validation-rmse:6.60479
[30]	validation-rmse:6.59670
[31]	validation-rmse:6.58906
[32]	validation-rmse:6.58019
[33]	validation-rmse:6.57299
[34]	validation-rmse:

2025/05/17 09:01:12 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/17 09:01:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\mikes\Documents\STUDY\mlops-zoomcamp\mlops_zoomcamp_hw\venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [09:01:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/17 09:01:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [28]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        booster = xgb.train(
            params = params, 
            dtrain = train, 
            num_boost_round = 1000, 
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )
        y_pred = booster.predict(valid)
        rmse = np.sqrt( mean_squared_error(y_val, y_pred))
        print("rmse", rmse)
        mlflow.log_metric("rmse", rmse)
    
    return {
        'loss': rmse,
        'status': STATUS_OK
    }

## Logging models

In [50]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [51]:
import mlflow.xgboost

with mlflow.start_run():

    best_params = {
        'max_depth': 30,
        'learning_rate': 0.10027586753294233,
        'reg_lambda': 0.011074980286498087,
        'reg_alpha': 0.018788520719314586,
        'min_child_weight': 1.06,
        'objective': 'reg:linear',
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
            params = best_params, 
            dtrain = train, 
            num_boost_round = 1000, 
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )
    
    y_pred = booster.predict(valid)
    rmse = np.sqrt( mean_squared_error(y_val, y_pred))

    mlflow.log_metric("rmse", rmse)

    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact('models/preprocessor.b', artifact_path='models_pickle')

    booster.save_model('models/booster.bin')
    mlflow.log_artifact('models/booster.bin', artifact_path='models_pickle')
    
    mlflow.xgboost.log_model(
        booster,
        artifact_path="models_mlflow"
    )

c:\Users\mikes\Documents\STUDY\mlops-zoomcamp\mlops_zoomcamp_hw\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [15:36:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.40996
[1]	validation-rmse:10.71118
[2]	validation-rmse:10.10437
[3]	validation-rmse:9.58291
[4]	validation-rmse:9.13230
[5]	validation-rmse:8.74782
[6]	validation-rmse:8.41670
[7]	validation-rmse:8.13685
[8]	validation-rmse:7.89791
[9]	validation-rmse:7.69647
[10]	validation-rmse:7.52636
[11]	validation-rmse:7.38153
[12]	validation-rmse:7.25852
[13]	validation-rmse:7.15466
[14]	validation-rmse:7.06707
[15]	validation-rmse:6.99317
[16]	validation-rmse:6.93005
[17]	validation-rmse:6.87754
[18]	validation-rmse:6.83124
[19]	validation-rmse:6.79173
[20]	validation-rmse:6.75722
[21]	validation-rmse:6.72853
[22]	validation-rmse:6.70243
[23]	validation-rmse:6.67990
[24]	validation-rmse:6.66082
[25]	validation-rmse:6.64412
[26]	validation-rmse:6.62974
[27]	validation-rmse:6.61531
[28]	validation-rmse:6.60295
[29]	validation-rmse:6.59355
[30]	validation-rmse:6.58320
[31]	validation-rmse:6.57526
[32]	validation-rmse:6.56791
[33]	validation-rmse:6.56153
[34]	validation-rmse:

C:\Users\mikes\AppData\Local\Temp\ipykernel_54064\2491624226.py:35: UserWarning: [15:38:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  booster.save_model('models/booster.bin')
c:\Users\mikes\Documents\STUDY\mlops-zoomcamp\mlops_zoomcamp_hw\venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:38:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/18 15:39:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
import mlflow
from pyspark.sql.functions import struct, col
model_uri = 'runs:/1c343a3b4eb2431f8f362c18e328e006/models_mlflow'

# Load model as a Spark UDF. Override result_type if the model does not return double values.
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

# Predict on a Spark DataFrame.
df.withColumn('predictions', loaded_model(struct(*map(col, df.columns))))

In [56]:
model_uri = 'runs:/1c343a3b4eb2431f8f362c18e328e006/models_mlflow'
loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)

In [58]:
loaded_model.predict(X_val)

array([14.622031 ,  7.1634502, 15.5969515, ..., 13.483162 ,  6.6690516,
        8.180701 ], shape=(61921,), dtype=float32)

In [47]:
y_pred[0:10]

array([14.622031 ,  7.1634502, 15.5969515, 24.352861 ,  9.595731 ,
       17.11418  , 11.628514 ,  8.725436 ,  8.992133 , 18.969328 ],
      dtype=float32)

In [54]:
xgboost_model = mlflow.xgboost.load_model( 'runs:/da6671c16d49428db7882bde8b304f9b/models_mlflow')


In [55]:
xgboost_model.predict(valid)

array([14.622031 ,  7.1634502, 15.5969515, ..., 13.483162 ,  6.6690516,
        8.180701 ], shape=(61921,), dtype=float32)

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_lambda': hp.loguniform('reg_lambda', -5, -1),
    'reg_alpha': hp.loguniform('reg_alpha', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42 
}

best_result = fmin(fn=objective,
                   space=search_space,
                   algo=tpe.suggest,
                   max_evals=50,
                   trials=Trials())

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\mikes\Documents\STUDY\mlops-zoomcamp\mlops_zoomcamp_hw\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [09:01:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.84304                          
[1]	validation-rmse:9.78834                           
[2]	validation-rmse:8.98022                           
[3]	validation-rmse:8.37552                           
[4]	validation-rmse:7.92359                           
[5]	validation-rmse:7.58966                           
[6]	validation-rmse:7.34195                           
[7]	validation-rmse:7.16039                           
[8]	validation-rmse:7.02962                           
[9]	validation-rmse:6.92676                           
[10]	validation-rmse:6.85220                          
[11]	validation-rmse:6.79411                          
[12]	validation-rmse:6.74752                          
[13]	validation-rmse:6.71451                          
[14]	validation-rmse:6.68689                          
[15]	validation-rmse:6.66446                          
[16]	validation-rmse:6.64872                          
[17]	validation-rmse:6.63663                          
[18]	valid

KeyboardInterrupt: 